In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [2]:
def load_data(file_path):
    """
    Load data from a CSV file.
    """
    return pd.read_csv(file_path)

In [3]:
def split_inputs_output(data, inputs, output):
    """
    Split features and target variables.
    """
    x = data[inputs]
    y = data[output]
    return x, y

In [4]:
def split_train_test(x, y, test_size=0.25, random_state=0):
    """
    Split data into train and test sets.
    """
    return train_test_split(x, y, test_size=test_size, random_state=random_state)

In [5]:
def preprocess(x_train, x_test):
    """
    Vectorize the text data.
    """
    vectorizer = CountVectorizer()
    x_train_cleaned = vectorizer.fit_transform(x_train)
    x_test_cleaned = vectorizer.transform(x_test)
    return x_train_cleaned, x_test_cleaned

In [6]:
def train_model(x_train_cleaned, y_train, hyperparameters):
    """
    Training the machine learning model.
    """
    clf = LogisticRegression()
    grid_search = GridSearchCV(clf, hyperparameters, cv=5)
    grid_search.fit(x_train_cleaned, y_train)
    return grid_search

In [7]:
def evaluate_model(model, x_train_cleaned, y_train, x_test_cleaned, y_test):
    """
    Evaluating the model.
    """
    y_train_pred = model.predict(x_train_cleaned)
    y_test_pred = model.predict(x_test_cleaned)

    train_score = metrics.accuracy_score(y_train, y_train_pred)
    test_score = metrics.accuracy_score(y_test, y_test_pred)
    
    return train_score, test_score

In [8]:
def workflow(data_path):
    DATA_PATH = data_path
    INPUTS = "Review text"
    OUTPUT = 'Ratings'
    HYPERPARAMETERS = {
        'C': [10],
        'max_iter': [100]
    }
    
    # Load data
    yonex_data = load_data(DATA_PATH)

    # Identify Inputs and Output
    x, y = split_inputs_output(yonex_data, INPUTS, OUTPUT)

    # Split data into train and test sets
    x_train, x_test, y_train, y_test = split_train_test(x, y)

    # Preprocess the data
    x_train_cleaned, x_test_cleaned = preprocess(x_train, x_test)

    # Build a model
    model = train_model(x_train_cleaned, y_train, HYPERPARAMETERS)
    
    # Evaluation
    train_score, test_score = evaluate_model(model, x_train_cleaned, y_train, x_test_cleaned, y_test)
    
    print("Train Score:", train_score)
    print("Test Score:", test_score)

In [9]:
if __name__ == "__main__":
    workflow(data_path="data/yonex_data.csv")

Train Score: 0.9573801316201818
Test Score: 0.8966165413533834


### Building a Prefect workflow

In [10]:
from prefect import task, flow

In [11]:
@task
def load_data(file_path):
    """
    Load data from a CSV file.
    """
    return pd.read_csv(file_path)

@task
def split_inputs_output(data, inputs, output):
    """
    Split features and target variables.
    """
    x = data[inputs]
    y = data[output]
    return x, y

@task
def split_train_test(x, y, test_size=0.25, random_state=0):
    """
    Split data into train and test sets.
    """
    return train_test_split(x, y, test_size=test_size, random_state=random_state)

@task
def preprocess(x_train, x_test):
    """
    Vectorize the text data.
    """
    vectorizer = CountVectorizer()
    x_train_cleaned = vectorizer.fit_transform(x_train)
    x_test_cleaned = vectorizer.transform(x_test)
    return x_train_cleaned, x_test_cleaned

@task
def train_model(x_train_cleaned, y_train, hyperparameters):
    """
    Training the machine learning model.
    """
    clf = LogisticRegression()
    grid_search = GridSearchCV(clf, hyperparameters, cv=5)
    grid_search.fit(x_train_cleaned, y_train)
    return grid_search

@task
def evaluate_model(model, x_train_cleaned, y_train, x_test_cleaned, y_test):
    """
    Evaluating the model.
    """
    y_train_pred = model.predict(x_train_cleaned)
    y_test_pred = model.predict(x_test_cleaned)

    train_score = metrics.accuracy_score(y_train, y_train_pred)
    test_score = metrics.accuracy_score(y_test, y_test_pred)
    
    return train_score, test_score

In [12]:
@flow(name="Logistic Regression Workflow")
def workflow():
    DATA_PATH = "data/yonex_data.csv"
    INPUTS = "Review text"
    OUTPUT = 'Ratings'
    HYPERPARAMETERS = {
        'C': [10],
        'max_iter': [100]
    }
    
    # Load data
    yonex_data = load_data(DATA_PATH)

    # Identify Inputs and Output
    x, y = split_inputs_output(yonex_data, INPUTS, OUTPUT)

    # Split data into train and test sets
    x_train, x_test, y_train, y_test = split_train_test(x, y)

    # Preprocess the data
    x_train_cleaned, x_test_cleaned = preprocess(x_train, x_test)

    # Build a model
    model = train_model(x_train_cleaned, y_train, HYPERPARAMETERS)
    
    # Evaluation
    train_score, test_score = evaluate_model(model, x_train_cleaned, y_train, x_test_cleaned, y_test)
    
    print("Train Score:", train_score)
    print("Test Score:", test_score)

In [13]:
if __name__=='__main__':
    workflow()

16:26:34.745 | INFO    | prefect.engine - Created flow run 'persimmon-seal' for flow 'Logistic Regression Workflow'

16:26:34.924 | INFO    | Flow run 'persimmon-seal' - Created task run 'load_data-0' for task 'load_data'

16:26:34.929 | INFO    | Flow run 'persimmon-seal' - Executing 'load_data-0' immediately...

16:26:35.094 | INFO    | Task run 'load_data-0' - Finished in state Completed()

16:26:35.158 | INFO    | Flow run 'persimmon-seal' - Created task run 'split_inputs_output-0' for task 'split_inputs_output'

...

16:26:36.909 | INFO    | Flow run 'persimmon-seal' - Created task run 'evaluate_model-0' for task 'evaluate_model'

16:26:36.909 | INFO    | Flow run 'persimmon-seal' - Executing 'evaluate_model-0' immediately...

16:26:37.193 | INFO    | Task run 'evaluate_model-0' - Finished in state Completed()

Train Score: 0.9573801316201818
Test Score: 0.8966165413533834


16:26:37.279 | INFO    | Flow run 'persimmon-seal' - Finished in state Completed('All states completed.')